## Setup and Configuration

First, let's import the necessary libraries and configure AWS credentials.

In [ ]:
import sys
import os

# Add parent directory to path to import modules
sys.path.append(os.path.dirname(os.path.abspath('')))

import json
import boto3
from datetime import datetime
from botocore.exceptions import ClientError


# Always try to load from environment first
aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
aws_session_token = os.environ.get('AWS_SESSION_TOKEN')
aws_region = os.environ.get('AWS_REGION', os.environ.get('AWS_DEFAULT_REGION', 'us-east-1'))
bedrock_model_id = os.environ.get('BEDROCK_MODEL_ID', 'us.amazon.nova-pro-v1:0')
print(f"Loading configuration... {aws_region}, {bedrock_model_id}")
print(f"  AWS_ACCESS_KEY_ID env: {aws_access_key_id[:10] + '...' if aws_access_key_id else 'Not set'}")
      
# Try to import configuration as fallback
try:
    import config
    if aws_access_key_id is None:
        aws_region = getattr(config, 'aws_region', aws_region)
        bedrock_model_id = getattr(config, 'bedrock_model_id', bedrock_model_id)
        aws_access_key_id = getattr(config, 'aws_access_key_id', None)
        aws_secret_access_key = getattr(config, 'aws_secret_access_key', None)
        aws_session_token = getattr(config, 'aws_session_token', None)
        print("✓ Configuration loaded from config.py")
    else:
        print("✓ Configuration loaded from environment variables")
except ImportError:
    print("✓ Configuration loaded from environment variables (config.py not found)")

print(f"  AWS Region: {aws_region}")
print(f"  Model ID: {bedrock_model_id}")
print(f"  AWS_ACCESS_KEY_ID: {aws_access_key_id[:10] + '...' if aws_access_key_id else 'Not set'}")
print(f"  AWS_SECRET_ACCESS_KEY: {'Set' if aws_secret_access_key else 'Not set'}")
print(f"  AWS_SESSION_TOKEN: {'Set' if aws_session_token else 'Not set'}")

## Initialize Bedrock Client

Connect to Amazon Bedrock service to access the Nova Pro model.

In [7]:
# Initialize Bedrock Runtime client
try:
    if aws_access_key_id and aws_secret_access_key:
        # Use explicit credentials (with session token if available)
        client_kwargs = {
            'service_name': 'bedrock-runtime',
            'region_name': aws_region,
            'aws_access_key_id': aws_access_key_id,
            'aws_secret_access_key': aws_secret_access_key
        }
        if aws_session_token:
            client_kwargs['aws_session_token'] = aws_session_token
        
        bedrock_client = boto3.client(**client_kwargs)
    else:
        # Use default AWS credentials (from ~/.aws/credentials or environment)
        bedrock_client = boto3.client(
            service_name='bedrock-runtime',
            region_name=aws_region
        )
    
    model_id = bedrock_model_id
    print(f"✓ Bedrock client initialized")
    print(f"  Model: {model_id}")
    print(f"  Region: {aws_region}")
    
except Exception as e:
    print(f"✗ Error initializing Bedrock client: {str(e)}")
    print("  Please check your AWS credentials")
    print("  Set AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY, and AWS_SESSION_TOKEN environment variables")
    print("  Or configure ~/.aws/credentials file")

## Load Sample Medical Transcript

Load a sample transcript from ambient listening software. This is an unstructured narrative of a patient encounter.

In [8]:
# Load sample transcript
transcript_path = '../sample_data/sample_transcript.txt'

with open(transcript_path, 'r') as f:
    transcript = f.read()

print("Sample Medical Transcript:")
print("=" * 80)
print(transcript)
print("=" * 80)
print(f"\nTranscript length: {len(transcript)} characters")

## Generate SOAP Note with Bedrock Nova Pro

Use the Amazon Bedrock Nova Pro model to convert the unstructured transcript into a structured SOAP note.

In [9]:
def generate_soap_note(transcript_text):
    """
    Generate a SOAP note from medical transcript using Bedrock Nova Pro.
    
    Args:
        transcript_text (str): Medical transcript
        
    Returns:
        dict: SOAP note with subjective, objective, assessment, and plan sections
    """
    system_prompt = "I am an expert SOAP note generator. Given a medical transcript of inpatient or outpatient visits I can create a SOAP note."
    
    user_prompt = f"""Please generate a comprehensive SOAP note from the following medical transcript. 

Format your response as follows:

SUBJECTIVE:
[Patient's reported symptoms, history, and concerns - use paragraphs and bullet points as appropriate]

OBJECTIVE:
[Physical examination findings, vital signs, and test results - use structured format with clear organization]

ASSESSMENT:
[Clinical diagnosis and evaluation - use numbered list for multiple diagnoses]

PLAN:
[Treatment plan and recommendations - use numbered or bulleted list]

Medical Transcript:
{transcript_text}

Please provide the SOAP note in clear, well-formatted text suitable for clinical documentation."""
    
    # Prepare request for Nova Pro
    request_body = {
        "messages": [
            {
                "role": "user",
                "content": [
                    {"text": user_prompt}
                ]
            }
        ],
        "system": [
            {"text": system_prompt}
        ],
        "inferenceConfig": {
            "maxTokens": 2000,
            "temperature": 0.3,
            "topP": 0.9
        }
    }
    
    try:
        print("Calling Amazon Bedrock Nova Pro...")
        
        # Invoke the model
        response = bedrock_client.converse(
            modelId=model_id,
            messages=request_body["messages"],
            system=request_body["system"],
            inferenceConfig=request_body["inferenceConfig"]
        )
        
        # Extract response text
        response_text = response['output']['message']['content'][0]['text']
        print(f"✓ Received response ({len(response_text)} characters)")
        
        return response_text
        
    except ClientError as e:
        error_code = e.response['Error']['Code']
        error_message = e.response['Error']['Message']
        print(f"✗ Bedrock API Error: {error_code} - {error_message}")
        raise
    except Exception as e:
        print(f"✗ Error: {str(e)}")
        raise

# Generate SOAP note
soap_note_text = generate_soap_note(transcript)

print("\n" + "=" * 80)
print("GENERATED SOAP NOTE")
print("=" * 80)
print(soap_note_text)
print("=" * 80)

## Parse SOAP Sections

Extract the individual SOAP sections (Subjective, Objective, Assessment, Plan) from the generated text.

In [10]:
import re

def extract_soap_sections(soap_text):
    """
    Extract SOAP sections from formatted text.
    
    Args:
        soap_text (str): Full SOAP note text
        
    Returns:
        dict: Dictionary with subjective, objective, assessment, and plan sections
    """
    sections = {
        'subjective': '',
        'objective': '',
        'assessment': '',
        'plan': ''
    }
    
    # Patterns to match section headers
    patterns = {
        'subjective': r'(?:^|\n)\*\*SUBJECTIVE:?\*\*|(?:^|\n)SUBJECTIVE:',
        'objective': r'(?:^|\n)\*\*OBJECTIVE:?\*\*|(?:^|\n)OBJECTIVE:',
        'assessment': r'(?:^|\n)\*\*ASSESSMENT:?\*\*|(?:^|\n)ASSESSMENT:',
        'plan': r'(?:^|\n)\*\*PLAN:?\*\*|(?:^|\n)PLAN:'
    }
    
    # Find all section positions
    positions = {}
    for section, pattern in patterns.items():
        match = re.search(pattern, soap_text, re.IGNORECASE | re.MULTILINE)
        if match:
            positions[section] = match.end()
    
    # Extract content between sections
    if positions:
        sorted_sections = sorted(positions.items(), key=lambda x: x[1])
        
        for i, (section, start_pos) in enumerate(sorted_sections):
            if i < len(sorted_sections) - 1:
                end_pos = sorted_sections[i + 1][1]
                content = soap_text[start_pos:end_pos]
            else:
                content = soap_text[start_pos:]
            
            # Clean up content
            content = re.sub(r'^[\s\n]+', '', content)
            content = re.sub(r'(?:^|\n)\*\*[A-Z]+:?\*\*.*$', '', content)
            content = content.strip()
            
            sections[section] = content
    
    return sections

# Parse sections
soap_sections = extract_soap_sections(soap_note_text)

# Display structured SOAP note
print("\nParsed SOAP Sections:")
print("=" * 80)

for section_name, section_content in soap_sections.items():
    print(f"\n{section_name.upper()}:")
    print("-" * 80)
    print(section_content if section_content else "(No content)")
    print()

print("=" * 80)

## Save SOAP Note

Save the generated SOAP note to a JSON file for use in subsequent steps.

In [11]:
# Create output structure
soap_output = {
    'subjective': soap_sections['subjective'],
    'objective': soap_sections['objective'],
    'assessment': soap_sections['assessment'],
    'plan': soap_sections['plan'],
    'generated_at': datetime.now().isoformat(),
    'source': 'bedrock_nova_pro',
    'model': model_id
}

# Save to file
output_file = 'soap_note_output.json'
with open(output_file, 'w') as f:
    json.dump(soap_output, f, indent=2)

print(f"✓ SOAP note saved to: {output_file}")
print(f"\nSummary:")
print(f"  Subjective: {len(soap_sections['subjective'])} characters")
print(f"  Objective: {len(soap_sections['objective'])} characters")
print(f"  Assessment: {len(soap_sections['assessment'])} characters")
print(f"  Plan: {len(soap_sections['plan'])} characters")

## Summary

### What We Accomplished

1. ✓ Loaded an unstructured medical transcript
2. ✓ Used Amazon Bedrock Nova Pro to generate a structured SOAP note
3. ✓ Parsed the SOAP note into four sections (S, O, A, P)
4. ✓ Saved the structured data for the next step

### Next Steps

The generated SOAP note will be used in **Step 2: Entity Extraction**, where we'll:
- Extract medical entities (problems, procedures, medications, labs)
- Extract context around each entity (200 characters)
- Use IMO Entity Extraction API for medical terminology recognition

### Key Takeaways

- **Amazon Bedrock Nova Pro** provides high-quality medical text understanding
- **SOAP format** transforms unstructured narratives into clinical documentation
- **Assessment and Plan** sections are critical for downstream entity extraction